In [1]:
import numpy as np

from keras.preprocessing import sequence
from keras.optimizers import SGD, RMSprop, Adagrad
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Embedding, RepeatVector, SharedDense
from keras.activations import time_distributed_softmax
from keras.layers.recurrent import LSTM, GRU
import theano
theano.config.optimizer='fast_compile'

In [ ]:
import json
filename = "/media/charles/data/articles"
h = open(filename)
all_jsons=[]
for line in h:    
    if line[0]=='[':
        all_jsons.append(line[:-1])
        
TitleList = []
#TextList = []
IndexList = []
count = 0

for oneJson in all_jsons:
    u = json.loads(oneJson)
    for item in u:
        fields = item['fields']
        TitleList.append(fields['title'].encode('ascii','ignore'))
        #TextList.append(fields['text'].encode('ascii','ignore'))
        IndexList.append(item['rowKey'].encode('ascii','ignore'))
        count+=1
        if count%10000==0:
            print("done: "+str(count))

all_jsons = []
del all_jsons

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.utils import np_utils

max_words = 10000
batch_size = 16

print "Loading data..."
X_seq = TitleList[0:60000]

print len(X_seq), 'total sequences'

print "Vectorizing sequence data..."
tokenizer = Tokenizer(nb_words=max_words)
tokenizer.fit_on_texts(X_seq, unknown_words=True)
X_seq = tokenizer.texts_to_sequences(X_seq, replace=True)

print "Divide train and test"
y_train = []
x_train = []
maxlen = 50

for s in X_seq:
    if len(s) < 3 or len(s)>maxlen:
        continue        
    x_train.append(s)
    y_train.append(s[::-1])


#Y_train = np_utils.to_categorical(y_train, max_words)
    
cut = int(len(X_seq)*0.8)
X_test = x_train[cut:]
X_train = x_train[:cut]
Y_test = y_train[cut:]
Y_train = y_train[:cut]
    
print "Pad sequences (samples x time)"
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
Y_train = sequence.pad_sequences(Y_train, maxlen=maxlen)
Y_test = sequence.pad_sequences(Y_test, maxlen=maxlen)

print 'X_train shape:', X_train.shape
print 'X_test shape:', X_test.shape

#print "Convert class vector to binary class matrix (for use with categorical_crossentropy)"
#Y_train = np_utils.to_categorical(y_train, nb_classes)
#Y_test = np_utils.to_categorical(y_test, nb_classes)
#print 'Y_train shape:', Y_train.shape
#print 'Y_test shape:', Y_test.shape


In [ ]:
voc_dic = tokenizer.word_index.items()
voc_dic.sort(key = lambda x: x[1])
voc = [x[0] for x in voc_dic]
inv_dic = {v:k for (k,v) in voc_dic}

In [ ]:
voc = np.asarray(voc)
np.savez_compressed('dataset',x_train=X_train,x_test=X_test,y_train=Y_train,y_test=Y_test,voc=voc)

In [2]:
x = np.load('dataset.npz')
print x.files
voc = x['voc'].tolist()
voc_dic = dict(zip(voc, range(len(voc))))
inv_dic = {v:k for (k,v) in voc_dic.items()}
X_train, X_test, Y_train, Y_test = x['x_train'],x['x_test'],x['y_train'],x['y_test']
max_words = max(np.max(X_train), np.max(X_test))+1
print max_words

['x_test', 'x_train', 'y_train', 'y_test', 'voc']
10000


In [5]:
max_features= max_words
emb_size = 256
lstm_size = 128
batch_size = 16
max_size = X_train.shape[1]

print 'Build model...'
model = Sequential()
model.add(Embedding(max_features, emb_size))
model.add(LSTM(emb_size, lstm_size)) # try using a GRU instead, for fun
model.add(RepeatVector(max_size))
#model.add(Dropout(0.5))
model.add(LSTM(lstm_size, lstm_size, return_sequences=True))
#model.add(Activation('time_distributed_softmax'))
model.add(SharedDense(lstm_size, max_features, activation='softmax'))
optimizer = SGD(lr=0.7/batch_size)


model.compile(loss='categorical_scalar_crossentropy', optimizer=optimizer)
# try using different optimizers and different optimizer configs


Build model...


In [6]:

print "Train..."
model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=10, verbose=1)
score = model.evaluate(X_test, Y_test, batch_size=batch_size)
print 'Test score:', score

classes = model.predict_classes(X_test, batch_size=batch_size)
acc = np_utils.accuracy(classes, Y_test)
print 'Test accuracy:', acc



Train...


KeyboardInterrupt: 

In [ ]:
def tostr(u):
    st = ""
    for x in u:
        st+=inv_dic[x]+" "
    return st.strip()

def see(X, Y):
    for x,y in zip (X,Y):
        print tostr(x), " / ", tostr(y)

def seeModel(X, model):
    see(X, model.predict_classes(X))

In [ ]:
seeModel(X_test[0:20], model)

In [ ]:
#x = model._predict(X_test[0:16])
x = model.predict_proba(X_test[0:16], batch_size=batch_size)

In [ ]:
model.predict_classes(X_test[0:1])

In [ ]:
tostr(X_test[0])

In [ ]:
(16,) + x[1:]